# Project: Train a Quadcopter How to Fly

Design an agent to fly a quadcopter, and then train it using a reinforcement learning algorithm of your choice! 

Try to apply the techniques you have learnt, but also feel free to come up with innovative ideas and test them.

## Instructions

Take a look at the files in the directory to better understand the structure of the project. 

- `task.py`: Define your task (environment) in this file.
- `agents/`: Folder containing reinforcement learning agents.
    - `policy_search.py`: A sample agent has been provided here.
    - `agent.py`: Develop your agent here.
- `physics_sim.py`: This file contains the simulator for the quadcopter.  **DO NOT MODIFY THIS FILE**.

For this project, you will define your own task in `task.py`.  Although we have provided a example task to get you started, you are encouraged to change it.  Later in this notebook, you will learn more about how to amend this file.

You will also design a reinforcement learning agent in `agent.py` to complete your chosen task.  

You are welcome to create any additional files to help you to organize your code.  For instance, you may find it useful to define a `model.py` file defining any needed neural network architectures.

## Controlling the Quadcopter

We provide a sample agent in the code cell below to show you how to use the sim to control the quadcopter.  This agent is even simpler than the sample agent that you'll examine (in `agents/policy_search.py`) later in this notebook!

The agent controls the quadcopter by setting the revolutions per second on each of its four rotors.  The provided agent in the `Basic_Agent` class below always selects a random action for each of the four rotors.  These four speeds are returned by the `act` method as a list of four floating-point numbers.  

For this project, the agent that you will implement in `agents/agent.py` will have a far more intelligent method for selecting actions!

In [3]:
import random

class Basic_Agent():
    def __init__(self, task):
        self.task = task
    
    def act(self):
        new_thrust = random.gauss(450., 25.)
        return [new_thrust + random.gauss(0., 1.) for x in range(4)]

Run the code cell below to have the agent select actions to control the quadcopter.  

Feel free to change the provided values of `runtime`, `init_pose`, `init_velocities`, and `init_angle_velocities` below to change the starting conditions of the quadcopter.

The `labels` list below annotates statistics that are saved while running the simulation.  All of this information is saved in a text file `data.txt` and stored in the dictionary `results`.  

In [4]:
%load_ext autoreload
%autoreload 2

import csv
import numpy as np
from task import Task

# Modify the values below to give the quadcopter a different starting position.
runtime = 5.                                     # time limit of the episode
init_pose = np.array([0., 0., 10., 0., 0., 0.])  # initial pose
init_velocities = np.array([0., 0., 0.])         # initial velocities
init_angle_velocities = np.array([0., 0., 0.])   # initial angle velocities
file_output = 'data.txt'                         # file name for saved results

# Setup
task = Task(init_pose, init_velocities, init_angle_velocities, runtime)
agent = Basic_Agent(task)
done = False
labels = ['time', 'x', 'y', 'z', 'phi', 'theta', 'psi', 'x_velocity',
          'y_velocity', 'z_velocity', 'phi_velocity', 'theta_velocity',
          'psi_velocity', 'rotor_speed1', 'rotor_speed2', 'rotor_speed3', 'rotor_speed4']
results = {x : [] for x in labels}

# Run the simulation, and save the results.
with open(file_output, 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(labels)
    while True:
        rotor_speeds = agent.act()
        _, _, done = task.step(rotor_speeds)
        to_write = [task.sim.time] + list(task.sim.pose) + list(task.sim.v) + list(task.sim.angular_v) + list(rotor_speeds)
        for ii in range(len(labels)):
            results[labels[ii]].append(to_write[ii])
        writer.writerow(to_write)
        if done:
            break

TypeError: output must be an array

Run the code cell below to visualize how the position of the quadcopter evolved during the simulation.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(results['time'], results['x'], label='x')
plt.plot(results['time'], results['y'], label='y')
plt.plot(results['time'], results['z'], label='z')
plt.legend()
_ = plt.ylim()

The next code cell visualizes the velocity of the quadcopter.

In [ ]:
plt.plot(results['time'], results['x_velocity'], label='x_hat')
plt.plot(results['time'], results['y_velocity'], label='y_hat')
plt.plot(results['time'], results['z_velocity'], label='z_hat')
plt.legend()
_ = plt.ylim()

Next, you can plot the Euler angles (the rotation of the quadcopter over the $x$-, $y$-, and $z$-axes),

In [ ]:
plt.plot(results['time'], results['phi'], label='phi')
plt.plot(results['time'], results['theta'], label='theta')
plt.plot(results['time'], results['psi'], label='psi')
plt.legend()
_ = plt.ylim()

before plotting the velocities (in radians per second) corresponding to each of the Euler angles.

In [ ]:
plt.plot(results['time'], results['phi_velocity'], label='phi_velocity')
plt.plot(results['time'], results['theta_velocity'], label='theta_velocity')
plt.plot(results['time'], results['psi_velocity'], label='psi_velocity')
plt.legend()
_ = plt.ylim()

Finally, you can use the code cell below to print the agent's choice of actions.  

In [ ]:
plt.plot(results['time'], results['rotor_speed1'], label='Rotor 1 revolutions / second')
plt.plot(results['time'], results['rotor_speed2'], label='Rotor 2 revolutions / second')
plt.plot(results['time'], results['rotor_speed3'], label='Rotor 3 revolutions / second')
plt.plot(results['time'], results['rotor_speed4'], label='Rotor 4 revolutions / second')
plt.legend()
_ = plt.ylim()

When specifying a task, you will derive the environment state from the simulator.  Run the code cell below to print the values of the following variables at the end of the simulation:
- `task.sim.pose` (the position of the quadcopter in ($x,y,z$) dimensions and the Euler angles),
- `task.sim.v` (the velocity of the quadcopter in ($x,y,z$) dimensions), and
- `task.sim.angular_v` (radians/second for each of the three Euler angles).

In [ ]:
# the pose, velocity, and angular velocity of the quadcopter at the end of the episode
print(task.sim.pose)
print(task.sim.v)
print(task.sim.angular_v)

In the sample task in `task.py`, we use the 6-dimensional pose of the quadcopter to construct the state of the environment at each timestep.  However, when amending the task for your purposes, you are welcome to expand the size of the state vector by including the velocity information.  You can use any combination of the pose, velocity, and angular velocity - feel free to tinker here, and construct the state to suit your task.

## The Task

A sample task has been provided for you in `task.py`.  Open this file in a new window now. 

The `__init__()` method is used to initialize several variables that are needed to specify the task.  
- The simulator is initialized as an instance of the `PhysicsSim` class (from `physics_sim.py`).  
- Inspired by the methodology in the original DDPG paper, we make use of action repeats.  For each timestep of the agent, we step the simulation `action_repeats` timesteps.  If you are not familiar with action repeats, please read the **Results** section in [the DDPG paper](https://arxiv.org/abs/1509.02971).
- We set the number of elements in the state vector.  For the sample task, we only work with the 6-dimensional pose information.  To set the size of the state (`state_size`), we must take action repeats into account.  
- The environment will always have a 4-dimensional action space, with one entry for each rotor (`action_size=4`). You can set the minimum (`action_low`) and maximum (`action_high`) values of each entry here.
- The sample task in this provided file is for the agent to reach a target position.  We specify that target position as a variable.

The `reset()` method resets the simulator.  The agent should call this method every time the episode ends.  You can see an example of this in the code cell below.

The `step()` method is perhaps the most important.  It accepts the agent's choice of action `rotor_speeds`, which is used to prepare the next state to pass on to the agent.  Then, the reward is computed from `get_reward()`.  The episode is considered done if the time limit has been exceeded, or the quadcopter has travelled outside of the bounds of the simulation.

In the next section, you will learn how to test the performance of an agent on this task.

## The Agent

The sample agent given in `agents/policy_search.py` uses a very simplistic linear policy to directly compute the action vector as a dot product of the state vector and a matrix of weights. Then, it randomly perturbs the parameters by adding some Gaussian noise, to produce a different policy. Based on the average reward obtained in each episode (`score`), it keeps track of the best set of parameters found so far, how the score is changing, and accordingly tweaks a scaling factor to widen or tighten the noise.

Run the code cell below to see how the agent performs on the sample task.

In [5]:
import sys
import pandas as pd
from agents.policy_search import PolicySearch_Agent
from task import Task

num_episodes = 1000
target_pos = np.array([0., 0., 10.])
task = Task(target_pos=target_pos)
agent = PolicySearch_Agent(task) 

for i_episode in range(1, num_episodes+1):
    state = agent.reset_episode() # start a new episode
    while True:
        action = agent.act(state) 
        next_state, reward, done = task.step(action)
        agent.step(reward, done)
        state = next_state
        if done:
            print("\rEpisode = {:4d}, score = {:7.3f} (best = {:7.3f}), noise_scale = {}".format(
                i_episode, agent.score, agent.best_score, agent.noise_scale), end="")  # [debug]
            break
    sys.stdout.flush()

ModuleNotFoundError: No module named 'agents'

This agent should perform very poorly on this task.  And that's where you come in!

## Define the Task, Design the Agent, and Train Your Agent!

Amend `task.py` to specify a task of your choosing.  If you're unsure what kind of task to specify, you may like to teach your quadcopter to takeoff, hover in place, land softly, or reach a target pose.  

After specifying your task, use the sample agent in `agents/policy_search.py` as a template to define your own agent in `agents/agent.py`.  You can borrow whatever you need from the sample agent, including ideas on how you might modularize your code (using helper methods like `act()`, `learn()`, `reset_episode()`, etc.).

Note that it is **highly unlikely** that the first agent and task that you specify will learn well.  You will likely have to tweak various hyperparameters and the reward function for your task until you arrive at reasonably good behavior.

As you develop your agent, it's important to keep an eye on how it's performing. Use the code above as inspiration to build in a mechanism to log/save the total rewards obtained in each episode to file.  If the episode rewards are gradually increasing, this is an indication that your agent is learning.

In [4]:

agent.actor.model.predict(np.array(np.reshape([ 0.,  0., 10.,  0.,  0.,  0.], [-1,6])))

In [ ]:
print(agent.task.sim.next_timestep([[50,75,80,98]]))

In [1]:
import numpy as np
from agent import Agent
agent = Agent(init_pose=[15,15,15,0,0,0], target_pos=[0,0,120], runtime=15)
agent.step()
for _ in range(10000):
    agent.step()
    len(agent.memory.memory)

Using TensorFlow backend.


Velocity
[0.        0.        0.5272287]
107.12142642814275 107.11625855828989 -0.005167869852868989
Velocity
[ 0.52024566 -0.12779885 -0.15815895]
107.11625855828989 107.11319067385797 -0.0030678844319140808
Velocity
[-0.19549455 -0.07666548 -0.25082684]
107.11319067385797 107.11736819813189 0.004177524273913491
Velocity
[ 0.49571155  0.06594561 -0.2811502 ]
107.11736819813189 107.12298823100967 0.00562003287778623
Velocity
[-0.19621483 -0.10779293 -0.61108643]
107.12298823100967 107.13209494578874 0.009106714779065328
Velocity
[ 0.5283079  -0.14861419 -0.8249232 ]
107.13209494578874 107.14627712654195 0.01418218075320965
Velocity
[ 0.10810627  0.42943142 -0.90184729]
107.14627712654195 107.16448780134947 0.018210674807519922
Velocity
[-0.31583724 -0.15983378 -1.0619629 ]
107.16448780134947 107.18382291589425 0.019335114544787757
Velocity
[ 0.22520788 -0.62548281 -1.11992247]
107.18382291589425 107.20398420378216 0.020161287887901835
Velocity
[ 0.88789557 -0.36391421 -1.45537324]
107.

Velocity
[-0.37057567  0.04984564  0.13362042]
107.10940349537462 107.10596951062435 -0.0034339847502735665
Velocity
[-0.46285066  0.07047652  0.12643868]
107.10596951062435 107.10242344218966 -0.0035460684346872995
Velocity
[-0.56158928  0.09532386  0.11331881]
107.10242344218966 107.09887381137814 -0.0035496308115199326
Velocity
[-0.66666608  0.12444927  0.09403265]
107.09887381137814 107.09543385755762 -0.0034399538205178715
Velocity
[-0.77794941  0.15777254  0.06830654]
107.09543385755762 107.09222226377285 -0.0032115937847692066
Velocity
[-0.89532977  0.19517665  0.03587957]
107.09222226377285 107.0893632357816 -0.0028590279912492633
Velocity
[-1.01865085  0.23639667 -0.00352913]
107.0893632357816 107.08698638034119 -0.0023768554404171027
Velocity
[-1.14783368  0.28115112 -0.05007233]
107.08698638034119 107.08522540484421 -0.0017609754969782898
Velocity
[-1.28255693  0.32906126 -0.10415608]
107.08522540484421 107.08421944711414 -0.0010059577300722822
Velocity
[-1.42260897  0.37972

Velocity
[ -8.23047427  -3.2227286  -10.52421051]
115.74613934630884 115.94119129325203 0.19505194694319528
Velocity
[ -8.22287746  -3.21980828 -10.70993661]
115.94119129325203 116.14021280666024 0.19902151340821206
Velocity
[ -8.21822755  -3.21775944 -10.90000924]
116.14021280666024 116.34329037154912 0.20307756488887208
Velocity
[ -8.21608983  -3.21661654 -11.09377329]
116.34329037154912 116.55049965401669 0.20720928246757353
Velocity
[ -8.21616629  -3.2164146  -11.29086585]
116.55049965401669 116.76190663946717 0.21140698545048053
Velocity
[ -8.21797712  -3.21712455 -11.49080195]
116.76190663946717 116.97756934352705 0.21566270405988064
Velocity
[ -8.22096361  -3.21867752 -11.69312096]
116.97756934352705 117.1975369416414 0.2199675981143514
Velocity
[ -8.22455721  -3.22098837 -11.89745925]
117.1975369416414 117.4218509659305 0.2243140242891002
Velocity
[ -8.22824287  -3.22397991 -12.10359668]
117.4218509659305 117.65054760478539 0.22869663885488478
Velocity
[ -8.23133362  -3.2275262

Velocity
[ 0.10217609 -0.16192318 -0.26538182]
107.29321659553904 107.29821781824602 0.005001222706980002
Velocity
[ 0.10697359 -0.167154   -0.26913845]
107.29821781824602 107.30329286741042 0.005075049164403822
Velocity
[ 0.11189862 -0.17246484 -0.27281986]
107.30329286741042 107.3084399606286 0.0051470932181842954
Velocity
[ 0.11694167 -0.17785153 -0.27664843]
107.3084399606286 107.31365994738911 0.005219986760508277
Velocity
[ 0.12209648 -0.18332314 -0.28050138]
107.31365994738911 107.31895462561567 0.005294678226562155
Velocity
[ 0.12736244 -0.18888568 -0.28425982]
107.31895462561567 107.32432339452201 0.005368768906336641
Velocity
[ 0.13273368 -0.19454252 -0.28800137]
107.32432339452201 107.32976522776325 0.005441833241235372
Velocity
[ 0.13820954 -0.20030364 -0.29168671]
107.32976522776325 107.33527944772499 0.005514219961739286
Velocity
[ 0.14378589 -0.20616759 -0.29542463]
107.33527944772499 107.34086604057649 0.005586592851500427
Velocity
[ 0.14945768 -0.21213542 -0.29939194]


...

Velocity
[-0.07827908  0.08470822 27.02122601]
83.01568924920625 82.49407689297051 -0.5216123562357353
Velocity
[-0.09129126  0.08483371 27.08407249]
82.49407689297051 81.97141166105429 -0.5226652319162213
Velocity
[-0.10523541  0.08489764 27.1441875 ]
81.97141166105429 81.44775030748818 -0.5236613535661121
Velocity
[-0.12006997  0.08491139 27.20109595]
81.44775030748818 80.92315194150532 -0.5245983659828539
Velocity
[-0.13582381  0.08488    27.25540987]
80.92315194150532 80.39767436682365 -0.5254775746816733
Velocity
[-0.15246196  0.08481581 27.30695052]
80.39767436682365 79.87137121723136 -0.5263031495922945
Velocity
[-0.16997622  0.0847358  27.35584393]
79.87137121723136 79.34429674145568 -0.5270744757756773
Velocity
[-0.18838356  0.08465463 27.40264266]
79.34429674145568 78.81649866796947 -0.5277980734862098
Velocity
[-0.20762128  0.08457203 27.44696002]
78.81649866796947 78.28802322772016 -0.5284754402493093
Velocity
[-0.22770714  0.08448996 27.48925343]
78.28802322772016 77.75891

Velocity
[-3.58477953 -0.33402172 28.27601809]
40.592408323380205 40.074256552542124 -0.518151770838081
Velocity
[-3.65765241 -0.35218673 28.27296239]
40.074256552542124 39.5569815062028 -0.5172750463393214
Velocity
[-3.73113382 -0.37069573 28.26940071]
39.5569815062028 39.04062391350879 -0.516357592694014
Velocity
[-3.80521372 -0.38954655 28.26531713]
39.04062391350879 38.52522864627562 -0.515395267233167
Velocity
[-3.87992068 -0.40874549 28.26081501]
38.52522864627562 38.01084178356847 -0.5143868627071484
Velocity
[-3.95530627 -0.4282987  28.25606378]
38.01084178356847 37.497508934677086 -0.5133328488913875
Velocity
[-4.03128315 -0.4481802  28.25083646]
37.497508934677086 36.98527874587153 -0.5122301888055532
Velocity
[-4.10786974 -0.46838304 28.24521163]
36.98527874587153 36.47420399252402 -0.5110747533475148
Velocity
[-4.18508618 -0.48891117 28.23926096]
36.47420399252402 35.96433872946904 -0.5098652630549765
Velocity
[-4.26294773 -0.5097742  28.23303473]
35.96433872946904 35.45573

Velocity
[-11.37171951  -3.00381101  26.33110972]
17.06686459243178 17.377747440833094 0.3108828484013131
Velocity
[-11.47712214  -3.04761359  26.28487833]
17.377747440833094 17.700683337764627 0.3229358969315328
Velocity
[-11.58286339  -3.09155278  26.23794661]
17.700683337764627 18.035021108461116 0.3343377706964894
Velocity
[-11.68894755  -3.13563027  26.19033778]
18.035021108461116 18.380135029244112 0.3451139207829961
Velocity
[-11.79534167  -3.17983341  26.14200847]
18.380135029244112 18.73542587223094 0.3552908429868289
Velocity
[-11.90199396  -3.22413965  26.09288912]
18.73542587223094 19.100321190030822 0.36489531779988127
Velocity
[-12.00891406  -3.26855086  26.0430044 ]
19.100321190030822 19.47427605329813 0.37395486326730776
Velocity
[-12.11610307  -3.3130712   25.99236425]
19.47427605329813 19.856773514625857 0.382497461327727
Velocity
[-12.22363083  -3.35773487  25.94107954]
19.856773514625857 20.247324885385304 0.39055137075944657
Velocity
[-12.33128484  -3.40246374  25.

Velocity
[ 0.11916836 -0.14420588 26.89281628]
84.0271898080724 83.50741176328951 -0.5197780447828819
Velocity
[ 0.12161921 -0.14178127 26.96200983]
83.50741176328951 82.98649560798361 -0.5209161553058976
Velocity
[ 0.12399468 -0.13919049 27.02761205]
82.98649560798361 82.46452123430488 -0.5219743736787308
Velocity
[ 0.1262906  -0.13643908 27.08966156]
82.46452123430488 81.94156438661447 -0.5229568476904092
Velocity
[ 0.12851895 -0.13352543 27.14860717]
81.94156438661447 81.41769618104253 -0.5238682055719437
Velocity
[ 0.13068331 -0.13045266 27.20424136]
81.41769618104253 80.89298551133326 -0.5247106697092647
Velocity
[ 0.13279157 -0.12721959 27.256955  ]
80.89298551133326 80.36749960669496 -0.5254859046383018
Velocity
[ 0.13484617 -0.12381115 27.30695512]
80.36749960669496 79.84130007232815 -0.5261995343668104
Velocity
[ 0.13684679 -0.12021949 27.35412211]
79.84130007232815 79.31444782282506 -0.5268522495030936
Velocity
[ 0.13879665 -0.11643361 27.39878294]
79.31444782282506 78.787001

Velocity
[ 1.00097699  0.93105539 28.32257161]
40.29020655848571 39.83076400800885 -0.459442550476858
Velocity
[ 1.02522681  0.95527084 28.3268536 ]
39.83076400800885 39.37443644710777 -0.4563275609010802
Velocity
[ 1.04984445  0.97980926 28.33124554]
39.37443644710777 38.92134195622593 -0.4530944908818384
Velocity
[ 1.07478352  1.00462558 28.33543547]
38.92134195622593 38.47160610301724 -0.44973585320869347
Velocity
[ 1.10002369  1.0297036  28.33931898]
38.47160610301724 38.02536240785029 -0.44624369516694884
Velocity
[ 1.12560454  1.05507923 28.34314302]
38.02536240785029 37.58274855092934 -0.4426138569209499
Velocity
[ 1.15153712  1.08076471 28.34698891]
37.58274855092934 37.14390536048897 -0.438843190440366
Velocity
[ 1.17783399  1.10677874 28.35097686]
37.14390536048897 36.70897794081466 -0.4349274196743096
Velocity
[ 1.20446253  1.1330957  28.35493769]
36.70897794081466 36.278117568748506 -0.43086037206615657
Velocity
[ 1.23142505  1.15972469 28.35892869]
36.278117568748506 35.85

In [2]:
positions, actions, rewards = agent.play()
for i in range(len(positions)):
    print('Step: %d, action: %.1f, %.1f, %.1f, position: %.1f, %.1f, %.1f, reward: %.3f.' % 
          (i, actions[i][0], actions[i][1], actions[i][2], 
           positions[i][0], positions[i][1], positions[i][2], rewards[i]))

Velocity
[0.         0.         0.77782923]
26.272525397633277 107.11380218778513 80.84127679015185
Velocity
[0.         0.         1.55501799]
107.11380218778513 107.09093587737614 -0.022866310408986124
Velocity
[0.         0.         2.33038556]
107.09093587737614 107.05285202415685 -0.038083853219291086
Velocity
[0.         0.         3.10275959]
107.05285202415685 106.99959848394303 -0.053253540213816564
Velocity
[0.         0.         3.87098319]
106.99959848394303 106.93124620444341 -0.06835227949962075
Velocity
[0.         0.         4.63392186]
106.93124620444341 106.84788884981224 -0.08335735463117544
Velocity
[0.         0.         5.39047014]
106.84788884981224 106.74964229073665 -0.09824655907559077
Velocity
[0.         0.         6.13955797]
106.74964229073665 106.63664396604806 -0.11299832468858995
Velocity
[0.         0.         6.88015651]
106.63664396604806 106.50905212341328 -0.12759184263477152
Velocity
[0.         0.         7.61128363]
106.50905212341328 106.367044

Velocity
[ 0.          0.         28.17358101]
32.26415290532138 31.84175241290404 -0.4224004924173421
Velocity
[ 0.          0.         28.17380279]
31.84175241290404 31.42377466712138 -0.417977745782661
Velocity
[ 0.          0.         28.17401234]
31.42377466712138 31.01039893866004 -0.4133757284613395
Velocity
[ 0.          0.         28.17421034]
31.01039893866004 30.60181213283548 -0.40858680582455875
Velocity
[ 0.          0.         28.17439742]
30.60181213283548 30.19820902312096 -0.4036031097145205
Velocity
[ 0.          0.         28.17457419]
30.19820902312096 29.799792472668877 -0.39841655045208313
Velocity
[ 0.          0.         28.17474121]
29.799792472668877 29.406773640040512 -0.3930188326283641
Velocity
[ 0.          0.         28.17489902]
29.406773640040512 29.019372164852076 -0.3874014751884367
Velocity
[ 0.          0.         28.17504813]
29.019372164852076 28.637816328503153 -0.3815558363489231
Velocity
[ 0.          0.         28.17518901]
28.637816328503153

In [4]:
positions

[array([ 0,  0, 40,  0,  0,  0]),
 array([0.00000000e+00, 0.00000000e+00, 4.00071266e+01, 7.03527651e-03,
        1.17446338e-01, 0.00000000e+00]),
 array([-1.06466539e-03,  6.34796153e-05,  4.00284407e+01,  1.16342611e-02,
         2.26478175e-01,  0.00000000e+00]),
 array([-5.23161004e-03,  2.93302421e-04,  4.00636948e+01,  1.37819949e-02,
         3.25906598e-01,  0.00000000e+00]),
 array([-1.43349618e-02,  7.44169794e-04,  4.01124405e+01,  1.34684484e-02,
         4.14661982e-01,  0.00000000e+00]),
 array([-2.99731644e-02,  1.42441954e-03,  4.01740927e+01,  1.06892332e-02,
         4.91817545e-01,  0.00000000e+00]),
 array([-5.34917004e-02,  2.30049622e-03,  4.02479935e+01,  5.44594773e-03,
         5.56602226e-01,  0.00000000e+00]),
 array([-8.59782890e-02,  3.30249912e-03,  4.03334711e+01,  6.28093146e+00,
         6.08408766e-01,  0.00000000e+00]),
 array([-1.28388144e-01,  4.32898166e-03,  4.04300644e+01,  6.26755135e+00,
         5.73401090e-01,  0.00000000e+00]),
 array([-1.8

In [4]:

import numpy as np
from agent import Agent
agent = Agent(init_pose=[0,0,10,0,0,0])
positions, actions, rewards = agent.sample_play([900,900,900,900])


[array([ 0.        ,  0.        , 10.00777829,  0.        ,  0.        ,
         0.        ]),
 array([ 0.        ,  0.        , 10.03110676,  0.        ,  0.        ,
         0.        ]),
 array([ 0.       ,  0.       , 10.0699608,  0.       ,  0.       ,
         0.       ]),
 array([ 0.        ,  0.        , 10.12429225,  0.        ,  0.        ,
         0.        ]),
 array([ 0.        ,  0.        , 10.19402968,  0.        ,  0.        ,
         0.        ]),
 array([ 0.        ,  0.        , 10.27907873,  0.        ,  0.        ,
         0.        ]),
 array([ 0.        ,  0.        , 10.37932265,  0.        ,  0.        ,
         0.        ]),
 array([ 0.        ,  0.        , 10.49462293,  0.        ,  0.        ,
         0.        ]),
 array([ 0.        ,  0.        , 10.62482008,  0.        ,  0.        ,
         0.        ]),
 array([ 0.        ,  0.        , 10.76973448,  0.        ,  0.        ,
         0.        ]),
 array([ 0.       ,  0.       , 10.9291674,  0

## Plot the Rewards

Once you are satisfied with your performance, plot the episode rewards, either from a single run, or averaged over multiple runs. 

In [ ]:
## TODO: Plot the rewards.

## Reflections

**Question 1**: Describe the task that you specified in `task.py`.  How did you design the reward function?

**Answer**:

**Question 2**: Discuss your agent briefly, using the following questions as a guide:

- What learning algorithm(s) did you try? What worked best for you?
- What was your final choice of hyperparameters (such as $\alpha$, $\gamma$, $\epsilon$, etc.)?
- What neural network architecture did you use (if any)? Specify layers, sizes, activation functions, etc.

**Answer**:

**Question 3**: Using the episode rewards plot, discuss how the agent learned over time.

- Was it an easy task to learn or hard?
- Was there a gradual learning curve, or an aha moment?
- How good was the final performance of the agent? (e.g. mean rewards over the last 10 episodes)

**Answer**:

**Question 4**: Briefly summarize your experience working on this project. You can use the following prompts for ideas.

- What was the hardest part of the project? (e.g. getting started, plotting, specifying the task, etc.)
- Did you find anything interesting in how the quadcopter or your agent behaved?

**Answer**: